In [40]:
# Import necessary libraries for all three problem statements
import numpy as np
from time import time
import random
from queue import PriorityQueue

In [21]:
import numpy as np

class StateNode:
    def __init__(self, parent, state, path_cost, heuristic_cost, action=None):
        self.parent = parent
        self.state = state
        self.action = action
        self.path_cost = path_cost
        self.heuristic_cost = heuristic_cost
        self.cost = path_cost + heuristic_cost

    def __hash__(self):
        return hash(str(self.state.flatten()))

    def __str__(self):
        return str(self.state)

    def __ne__(self, other):
        return hash(''.join(self.state.flatten())) != hash(''.join(other.state.flatten()))

    def __eq__(self, other):
        return hash(''.join(self.state.flatten())) == hash(''.join(other.state.flatten()))

class PriorityQueue:
    def __init__(self):
        self.queue = []
        self.hashes = {}

    def push(self, node):
        node_hash = hash(node)
        if node_hash not in self.hashes:
            self.hashes[node_hash] = 1
            self.queue.append(node)

    def pop(self):
        min_cost_index = self._find_min_cost_index()
        return self.queue.pop(min_cost_index)

    def _find_min_cost_index(self):
        min_cost = 10**18
        min_index = -1
        for i, node in enumerate(self.queue):
            if node.cost < min_cost:
                min_cost = node.cost
                min_index = i
        return min_index

    def is_empty(self):
        return not bool(self.queue)

    def __str__(self):
        return str([node.state for node in self.queue])

    def __len__(self):
        return len(self.queue)

class BoardEnvironment:
    def __init__(self, start_state=None, goal_state=None):
        self.actions = [1, 2, 3, 4]  # 1 - Up, 2 - Down, 3 - Right, 4 - Left
        self.goal_state = goal_state if goal_state is not None else self.generate_goal_state()
        self.start_state = start_state if start_state is not None else self.generate_start_state()

    def generate_state(self, center_val):
        state = np.zeros((7, 7))
        state[[0, 1, 5, 6], :] = -1
        state[:, [0, 1, 5, 6]] = -1
        state[2:5, 2:5] = 1
        state[3, 3] = center_val
        return state

    def generate_start_state(self):
        return self.generate_state(0)

    def generate_goal_state(self):
        return self.generate_state(1)

    def get_start_state(self):
        return self.start_state

    def get_goal_state(self):
        return self.goal_state

    def get_next_states(self, state):
        new_states = []
        spaces = [(i, j) for i in range(7) for j in range(7) if state[i][j] == 0]
        for x, y in spaces:
            new_states.extend(self.get_new_states_for_space(state, x, y))
        return new_states

    def get_new_states_for_space(self, state, x, y):
        new_states = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        for dx, dy in directions:
            if self.can_move(state, x, y, dx, dy):
                new_state = state.copy()
                new_state[x][y] = 1
                new_state[x + dx][y + dy] = 0
                new_state[x + 2 * dx][y + 2 * dy] = 0
                action = f'({x + 2 * dx}, {y + 2 * dy}) -> ({x}, {y})'
                new_states.append((new_state, action))
        return new_states

    def can_move(self, state, x, y, dx, dy):
        return (0 <= x + 2 * dx < 7 and 0 <= y + 2 * dy < 7 and
                state[x + dx][y + dy] == 1 and state[x + 2 * dx][y + 2 * dy] == 1)

    def reached_goal(self, state):
        return np.array_equal(state, self.goal_state)


In [22]:
import time

class FirstPlayer:
    def __init__(self, environment, heuristic_function):
        self.frontier = PriorityQueue()
        self.explored = {}
        self.environment = environment
        self.heuristic = heuristic_function
        self.start_state = self.environment.get_start_state()
        self.goal_state = self.environment.get_goal_state()
        self.goal_node = None

    def run_search(self):
        self.initialize_frontier()
        start_time = time.time()
        while not self.frontier.is_empty():
            current_node = self.frontier.pop()
            if self.is_explored(current_node):
                continue
            self.add_to_explored(current_node)
            if self.environment.reached_goal(current_node.state):
                print("Goal reached!")
                self.goal_node = current_node
                break
            self.expand_frontier(current_node)
        end_time = time.time()
        print(end_time - start_time)
        return end_time - start_time

    def initialize_frontier(self):
        initial_node = StateNode(parent=None, state=self.start_state, path_cost=0, heuristic_cost=0)
        self.frontier.push(initial_node)

    def is_explored(self, node):
        return hash(node) in self.explored

    def add_to_explored(self, node):
        self.explored[hash(node)] = node

    def expand_frontier(self, current_node):
        next_states = self.environment.get_next_states(current_node.state)
        for state in next_states:
            heuristic_cost = self.heuristic(state[0])
            node = StateNode(parent=current_node, state=state[0], path_cost=current_node.path_cost+1, heuristic_cost=heuristic_cost, action=state[1])
            self.frontier.push(node)

    def print_path(self):
        node = self.goal_node
        nodes = []
        while node is not None:
            nodes.append(node)
            node = node.parent
        for step, node in enumerate(reversed(nodes), start=1):
            print(f"Step: {step}")
            print(node.action)


In [23]:
def zero_heuristic(curr_state):
    return 0

def heuristic_one(curr_state):
    return sum(abs(i-3) + abs(j-3) for i in range(7) for j in range(7) if curr_state[i][j] == 1)

def heuristic_two(curr_state):
    return sum(2 ** max(abs(i - 3), abs(j - 3)) for i in range(7) for j in range(7) if curr_state[i][j] == 1)



In [24]:
total_time = 0
explored_nodes = []
frontier_nodes = []

for _ in range(5):
    agent = FirstPlayer(BoardEnvironment(), heuristic_two)
    total_time += agent.run_search()
    explored_nodes.append(len(agent.explored))
    frontier_nodes.append(len(agent.frontier))

print("Average time:", total_time / 5)
print("Average number of explored nodes:", sum(explored_nodes) / 5)
print("Average number of nodes in frontier:", sum(frontier_nodes) / 5)


0.002454042434692383
0.001295328140258789
0.0011262893676757812
0.0011415481567382812
0.0011200904846191406
Average time: 0.001427459716796875
Average number of explored nodes: 1.0
Average number of nodes in frontier: 0.0


In [26]:
import time

class SecondPlayer:
    def __init__(self, environment, heuristic_function):
        self.frontier = PriorityQueue()
        self.explored = {}
        self.environment = environment
        self.heuristic = heuristic_function
        self.start_state = self.environment.get_start_state()
        self.goal_state = self.environment.get_goal_state()
        self.goal_node = None

    def run_search(self):
        self.initialize_frontier()
        start_time = time.time()
        while not self.frontier.is_empty():
            current_node = self.frontier.pop()
            if self.is_explored(current_node):
                continue
            self.add_to_explored(current_node)
            if self.environment.reached_goal(current_node.state):
                print("Goal reached!")
                self.goal_node = current_node
                break
            self.expand_frontier(current_node)
        end_time = time.time()
        print(end_time - start_time)
        return end_time - start_time

    def initialize_frontier(self):
        initial_node = StateNode(parent=None, state=self.start_state, path_cost=0, heuristic_cost=0)
        self.frontier.push(initial_node)

    def is_explored(self, node):
        return hash(node) in self.explored

    def add_to_explored(self, node):
        self.explored[hash(node)] = node

    def expand_frontier(self, current_node):
        next_states = self.environment.get_next_states(current_node.state)
        for state in next_states:
            heuristic_cost = self.heuristic(state[0])
            node = StateNode(parent=current_node, state=state[0], path_cost=0, heuristic_cost=heuristic_cost, action=state[1])
            self.frontier.push(node)

    def print_path(self):
        node = self.goal_node
        nodes = []
        while node is not None:
            nodes.append(node)
            node = node.parent
        for step, node in enumerate(reversed(nodes), start=1):
            print(f"Step: {step}")
            print(node.action)


In [27]:
total_time = 0
explored_nodes = []
frontier_nodes = []

for _ in range(5):
    agent = SecondPlayer(BoardEnvironment(), heuristic_two)
    total_time += agent.run_search()
    explored_nodes.append(len(agent.explored))
    frontier_nodes.append(len(agent.frontier))

print("Average time:", total_time / 5)
print("Average number of explored nodes:", sum(explored_nodes) / 5)
print("Average number of nodes in frontier:", sum(frontier_nodes) / 5)


0.0012238025665283203
0.0014774799346923828
0.0011682510375976562
0.000997304916381836
0.0007135868072509766
Average time: 0.0011160850524902343
Average number of explored nodes: 1.0
Average number of nodes in frontier: 0.0


In [28]:
import random

def generate_instance(n, k, m):
    variables = [chr(i + 65) for i in range(n)]
    clauses = []

    for _ in range(m):
        clause = []
        for _ in range(k):
            x = random.choice(variables)
            variables.remove(x)
            clause.append('~' + x if random.random() < 0.5 else x)
        clauses.append(' or '.join(clause))
        variables.extend(clause)

    problem = ' and '.join(f'({clause})' for clause in clauses)
    return f'(({problem}))'

for i in range(10):
    print(f"Problem {i+1}: {generate_instance(12, 3, 4)}")


Problem 1: (((H or E or ~I) and (J or C or B) and (K or ~G or C) and (E or ~A or ~~G)))
Problem 2: (((G or ~E or I) and (~D or ~G or ~J) and (~~E or K or ~A) and (~K or ~~A or ~J)))
Problem 3: (((~C or H or ~L) and (A or ~B or H) and (~K or ~~L or ~H) and (~A or ~B or E)))
Problem 4: (((G or B or I) and (H or ~A or ~F) and (~~A or ~K or J) and (E or B or D)))
Problem 5: (((F or C or L) and (J or ~E or ~F) and (B or ~G or ~L) and (~K or ~I or ~E)))
Problem 6: (((F or ~G or C) and (~K or L or ~B) and (~F or ~A or ~~K) and (~~~K or ~~F or L)))
Problem 7: (((~L or G or ~C) and (~I or E or ~A) and (~A or K or B) and (~F or ~D or ~J)))
Problem 8: (((C or ~E or ~F) and (G or ~I or K) and (~J or L or H) and (C or K or ~~I)))
Problem 9: (((~F or ~A or H) and (~K or H or ~I) and (~~F or G or ~~K) and (~B or G or ~L)))
Problem 10: (((~C or ~G or A) and (K or ~C or A) and (F or ~L or H) and (~I or ~A or ~K)))


In [29]:
from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority_value: int
    item_value: Any=field(compare=False)


In [30]:
import random

def generate_instance(n_variables, k_literals, m_clauses):
    variables = [chr(i + 65) for i in range(n_variables)]
    problem_parts = []

    for _ in range(m_clauses):
        clause = []
        for _ in range(k_literals):
            x = random.choice(variables)
            variables.remove(x)
            clause.append('~' + x if random.random() < 0.5 else x)
        problem_parts.append(' or '.join(clause))
        variables.extend(clause)

    problem = ' and '.join(f'({part})' for part in problem_parts)
    return f'(({problem}))'


In [32]:
import random

def generate_random_assignment(num_variables):
    return [random.randint(0, 1) for _ in range(num_variables)]


In [31]:
def evaluate_fitness(assignment, k, variables, pos_or_neg):
    fitness = 0
    clause_eval = 0

    for i, var in enumerate(variables):
        clause_eval |= assignment[var] if pos_or_neg[i] == 'P' else 1 - assignment[var]
        if i % k == k - 1:
            fitness += clause_eval
            clause_eval = 0

    return fitness


In [33]:
def hill_climbing(assignment, depth, k, variables, pos_or_neg):
    for d in range(depth):
        current_fitness = evaluate_fitness(assignment, k, variables, pos_or_neg)

        if current_fitness == len(variables):
            return assignment

        change = None
        for c in assignment:
            neighbour = assignment.copy()
            neighbour[c] = 1 - neighbour[c]

            neighbour_fitness = evaluate_fitness(neighbour, k, variables, pos_or_neg)
            if neighbour_fitness > current_fitness:
                current_fitness = neighbour_fitness
                change = c

        if change is not None:
            assignment[change] = 1 - assignment[change]

    return assignment


In [51]:
from queue import PriorityQueue
def beam_search(assignment, k, variables, pos_or_neg, beam_width, steps):
    beam = PriorityQueue()
    current = assignment
    beam.put(PrioritizedItem(-evaluate_fitness(current, k, variables, pos_or_neg), assignment))

    for _ in range(steps):
        if beam.empty():
            break

        state = beam.get()
        current = state.item_value
        x = -state.priority_value

        if x == len(variables):
            return current

        for c in current:
            neighbour = current.copy()
            neighbour[c] = 1 - neighbour[c]
            neighbour_fitness = -evaluate_fitness(neighbour, k, variables, pos_or_neg)

            if beam.qsize() < beam_width or neighbour_fitness > beam.queue[0].priority_value:
                if beam.qsize() == beam_width:
                    beam.get()
                beam.put(PrioritizedItem(neighbour_fitness, neighbour))

    return current


In [46]:
import random

def neighbour1(assignment):
    c = random.choice(list(assignment))
    assignment[c] = 1 - assignment[c]
    return assignment

def neighbour2(assignment):
    c = random.choice(list(assignment))
    d = random.choice(list(assignment))
    while d == c:
        d = random.choice(list(assignment))
    x = assignment[c]
    assignment[c] = assignment[d]
    assignment[d] = x
    return assignment

def neighbour3(assignment):
    x = list(assignment.keys())[0]
    assignment[x] = 1 - assignment[x]
    return assignment


In [47]:
def variable_neighbourhood(assignment, k, variables, pos_or_neg, steps):
    s = 0
    current = assignment

    while s < steps:
        current = assignment
        x = evaluate_fitness(assignment, k, variables, pos_or_neg)
        if x == len(variables):
            return current

        nbr1 = neighbour1(current.copy())
        nbr2 = neighbour2(current.copy())
        nbr3 = neighbour3(current.copy())

        fn1 = evaluate_fitness(nbr1, k, variables, pos_or_neg)
        fn2 = evaluate_fitness(nbr2, k, variables, pos_or_neg)
        fn3 = evaluate_fitness(nbr3, k, variables, pos_or_neg)

        if max(fn1, fn2, fn3) > x:
            x = max(fn1, fn2, fn3)
            if x == fn1:
                current = nbr1
            elif x == fn2:
                current = nbr2
            else:
                current = nbr3
        s += 1

    return current


In [52]:
n = 25
k = 3
m = 1000
problem = generate_instance(n,k,m)
numVars = set()
variables = []
posOrNeg = []

prevNeg = False

for i in range(len(problem)):
  if problem[i] == '~':
    prevNeg = True
  elif ord(problem[i]) >= 65 and ord(problem[i]) <= 90:
    if prevNeg == True:
      posOrNeg.append('N')
      prevNeg = False
    else:
      posOrNeg.append('P')

    variables.append(problem[i])
    numVars.add(problem[i])


values = generate_random_assignment(len(numVars))
startState = dict()

index = 0
for c in numVars:
  startState[c] = values[index]
  index += 1

print(startState)
print("Fitness for Starting State: ", evaluate_fitness(startState, k, variables, posOrNeg))
solution  = hill_climbing(startState.copy(), 100, k, variables, posOrNeg)
print("Fitness for Hill Climbing Solution: ", evaluate_fitness(solution, k, variables, posOrNeg))
solution = beam_search(startState.copy(), k, variables, posOrNeg, 3, 1000)
print("Fitness for Beam Search Solution (Beam-Width = 3): ", evaluate_fitness(solution, k, variables, posOrNeg))
solution = beam_search(startState.copy(), k, variables, posOrNeg, 4, 1000)
print("Fitness for Beam Search Solution (Beam-Width = 4): ", evaluate_fitness(solution, k, variables, posOrNeg))


print("Neighbour 1: ", neighbour1(startState.copy()))
print("Neighbour 2: ", neighbour2(startState.copy()))
print("Neighbour 3: ", neighbour3(startState.copy()))

solution = variable_neighbourhood(startState.copy(), k, variables, posOrNeg, 1000)
print("Fitness for Variable-Neigbourhood-Descent: ", evaluate_fitness(solution, k, variables, posOrNeg))

{'O': 0, 'J': 0, 'B': 0, 'X': 0, 'R': 0, 'H': 1, 'S': 1, 'A': 1, 'K': 1, 'N': 1, 'C': 1, 'Y': 1, 'F': 0, 'E': 1, 'V': 0, 'G': 1, 'Q': 1, 'L': 1, 'T': 0, 'W': 0, 'U': 1, 'P': 0, 'D': 0, 'M': 1, 'I': 0}
Fitness for Starting State:  894
Fitness for Hill Climbing Solution:  1000
Fitness for Beam Search Solution (Beam-Width = 3):  15
Fitness for Beam Search Solution (Beam-Width = 4):  15
Neighbour 1:  {'O': 0, 'J': 0, 'B': 0, 'X': 0, 'R': 0, 'H': 1, 'S': 1, 'A': 1, 'K': 1, 'N': 1, 'C': 1, 'Y': 1, 'F': 0, 'E': 1, 'V': 0, 'G': 1, 'Q': 1, 'L': 1, 'T': 0, 'W': 0, 'U': 1, 'P': 0, 'D': 0, 'M': 1, 'I': 1}
Neighbour 2:  {'O': 0, 'J': 0, 'B': 0, 'X': 0, 'R': 0, 'H': 1, 'S': 1, 'A': 0, 'K': 1, 'N': 1, 'C': 1, 'Y': 1, 'F': 0, 'E': 1, 'V': 0, 'G': 1, 'Q': 1, 'L': 1, 'T': 1, 'W': 0, 'U': 1, 'P': 0, 'D': 0, 'M': 1, 'I': 0}
Neighbour 3:  {'O': 1, 'J': 0, 'B': 0, 'X': 0, 'R': 0, 'H': 1, 'S': 1, 'A': 1, 'K': 1, 'N': 1, 'C': 1, 'Y': 1, 'F': 0, 'E': 1, 'V': 0, 'G': 1, 'Q': 1, 'L': 1, 'T': 0, 'W': 0, 'U': 1, '